In [1]:
!pip install -U langchain-ollama langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [4]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [5]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [6]:
df.shape

(40000, 2)

In [7]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [8]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [9]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [10]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [11]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [12]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

CPU times: user 1.84 s, sys: 51.2 ms, total: 1.89 s
Wall time: 1.89 s


In [13]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [14]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

CPU times: user 2.05 s, sys: 51.9 ms, total: 2.1 s
Wall time: 2.13 s


In [15]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [16]:
coffee_reviews_df.shape

(4981, 2)

In [17]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [18]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [19]:
!ollama pull gemma3:270m

In [20]:
!ollama list

NAME           ID              SIZE      MODIFIED     
gemma3:270m    e7d36fb2c3b3    291 MB    1 second ago    


In [21]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ 😊



In [22]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T13:58:30.810623784Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [23]:
!pip install --upgrade langchain langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstallin

In [24]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate  # เปลี่ยนจาก langchain.prompts
from langchain_core.output_parsers import StrOutputParser

In [25]:
llm = OllamaLLM(model="gemma3:270m")

In [26]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ 😊



In [27]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [28]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [29]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [30]:
chain = prompt | llm

In [31]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [32]:
response

'แน่นอนครับ! ผมจะช่วยคุณเขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10 ครับ\n\n**โค้ด:**\n\n```python\ndef find_sum(numbers):\n  """\n  หาผลรวมของตัวเลข 1 ถึง 10\n  """\n  sum = 0\n  for number in numbers:\n    sum += number\n  return sum\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum(numbers)\nprint(result)\n```\n\n**คำอธิบายโค้ด:**\n\n1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะทำการค้นหาผลรวม\n2.  **`sum = 0`**: Initialize variable `sum` to 0\n3.  **`for number in numbers:`**:  ภายใน loop จะทำการคำนับค่าในแต่ละตัวเลขใน `numbers`\n4.  **`sum += number`**:  เมื่อค่า `number` ได้ค่าแล้ว ให้ทำการเพิ่มค่า `number` เข้าไปใน `sum`\n5.  **`return sum`**:  เมื่อ `sum` มีค่าแล้ว ให้คืนค่า `sum`\n6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้าง iterable ที่มีตัวเลข 1 ถึง 10\n7.  **`result = find_sum(numbers)`**:  เรียกใช้ `find_sum` เพื่อหาผลรวมของตัวเลข 1 ถึง 10\n8.  **`print(result)`**:  แสดงผลลัพธ์ที่ได้จาก `find_sum`\n\n**การใช

In [33]:
from IPython.display import display, Markdown


In [34]:
display(Markdown(response))

แน่นอนครับ! ผมจะช่วยคุณเขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10 ครับ

**โค้ด:**

```python
def find_sum(numbers):
  """
  หาผลรวมของตัวเลข 1 ถึง 10
  """
  sum = 0
  for number in numbers:
    sum += number
  return sum

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum(numbers)
print(result)
```

**คำอธิบายโค้ด:**

1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะทำการค้นหาผลรวม
2.  **`sum = 0`**: Initialize variable `sum` to 0
3.  **`for number in numbers:`**:  ภายใน loop จะทำการคำนับค่าในแต่ละตัวเลขใน `numbers`
4.  **`sum += number`**:  เมื่อค่า `number` ได้ค่าแล้ว ให้ทำการเพิ่มค่า `number` เข้าไปใน `sum`
5.  **`return sum`**:  เมื่อ `sum` มีค่าแล้ว ให้คืนค่า `sum`
6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้าง iterable ที่มีตัวเลข 1 ถึง 10
7.  **`result = find_sum(numbers)`**:  เรียกใช้ `find_sum` เพื่อหาผลรวมของตัวเลข 1 ถึง 10
8.  **`print(result)`**:  แสดงผลลัพธ์ที่ได้จาก `find_sum`

**การใช้งานโค้ด:**

```python
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum(numbers)
print(result)
```

**Output:**

```
8
```

**คำอธิบายโค้ด:**

*   ชื่อ function `find_sum` ที่กำหนด
*   ตัวอย่างการใช้งานที่เป็นไปได้
*   การเรียกใช้ `find_sum` เพื่อหาผลรวมของตัวเลข
*   การแสดงผลลัพธ์ที่ได้จาก `find_sum`

**ข้อควรระวัง:**

*   การเพิ่มค่า `number` เข้าไปใน `sum` จะทำให้ `sum` เพิ่มขึ้นด้วย
*   การคำนวณผลรวมของตัวเลขเป็นเรื่องที่ซับซ้อนและอาจต้องใช้โปรแกรมคอมพิวเตอร์ (เช่น Python) เพื่อคำนวณได้ถูกต้อง
*   การใช้ `sum` ในการคำนวณผลรวมของตัวเลขอาจทำให้เกิดข้อผิดพลาดได้

หวังว่าโค้ดนี้จะเป็นประโยชน์นะครับ! หากมีคำถามเพิ่มเติม ถามได้เลยครับ

In [35]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [36]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [37]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [38]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)
chain4 = prompt4 | llm

In [39]:
parser_chain = StrOutputParser()

In [40]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [41]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [42]:
result

'ถูกต้องแล้วค่ะ\n'

In [43]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.3 MB/s eta 0:00:00


In [44]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI
from google.colab import userdata


In [45]:
OPENAI_API_KEY = "xxx"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [46]:
llm2 = ChatOpenAI(model="gpt-4o-mini")

In [47]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [48]:
chain1 = prompt1 | llm2
chain2 = prompt2 | llm2
chain3 = prompt3 | llm2
chain4 = prompt4 | llm2

full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [53]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: xxx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [52]:
result

'ถูกต้องแล้วค่ะ\n'

In [54]:
formatted_result = result.replace('\\[', '$$') \
                         .replace('\\]', '$$') \
                         .replace('\\(', '$') \
                         .replace('\\)', '$')

In [55]:
display(Markdown(formatted_result))

ถูกต้องแล้วค่ะ


In [56]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [57]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [58]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [59]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [60]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [61]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [62]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [63]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = text.replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 26.6 ms, sys: 4.64 ms, total: 31.3 ms
Wall time: 4.37 s


In [64]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": ["ร้านอาหารใหญ่", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ขนมไทย", "ขนมหวาน", "ฟรี", "ราคา", "แพง", "เกิน", "รับไม่ไหว", "กินแล้ว", "ขนมหวาน", "ไทย"]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "กาแฟ",
    "ร้านวาวี",
    "อารีย์"
  ]
}
```



In [65]:
keyword_extract = keyword_prompt | llm2 | StrOutputParser()

In [66]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [67]:
!ollama list

NAME                                  ID              SIZE      MODIFIED       
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    14 seconds ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    6 minutes ago     


In [68]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b:latest")

In [69]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [70]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 107 ms, sys: 15.8 ms, total: 123 ms
Wall time: 17.5 s


In [71]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที่คุณระบุมาเพื่อดึงคำสำคัญจากรีวิวนี้:

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้พูดถึงประสบการณ์การรับประทานอาหารในร้านอาหารแห่งหนึ่ง โดยมีประเด็นหลักๆ ดังนี้:

*   ร้านใหญ่มาก
*   ห้องน้ำอยู่ก่อนถึงห้องอาหาร
*   อาหารบางอย่างราคาแพงเกินไป (เช่น น้ำผึ้ง, พะแนงห่อไข่)
*   อาหารบางอย่างอร่อย (เช่น ทอดมันหัวปลี)
*   มีขนมหวานให้ฟรี

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

หัวข้อหลักที่กล่าวถึงในรีวิว ได้แก่:

*   ขนาดร้าน
*   ราคาอาหาร
*   รสชาติอาหาร (อร่อย/ไม่น่าประทับใจ)
*   ขนมหวาน

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

จากหัวข้อหลัก ผมเลือกคำสำคัญ 3 คำ ได้แก่:

1.  ราคา (เนื่องจากมีการกล่าวถึงราคาอาหารที่แพง)
2.  รสชาติ (มีการบอกว่าอาหารบางอย่างอร่อย/ไม่น่าประทับใจ)
3.  ขนาด (ร้านใหญ่มาก)

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคา", "รสชาติ", "ขนาด"]
}
```

หวังว่าคำตอบนี้จะเป็นประโยชน์นะครับ หากคุณมีค

In [72]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [73]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [74]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [75]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 99.9 ms, sys: 11 ms, total: 111 ms
Wall time: 11.2 s


In [76]:
results[0]['text']['keywords']

['ร้านอาหาร', 'ราคา', 'อาหาร']

In [77]:
results[1]['text']['keywords']

['กาแฟ', 'บรรยากาศ', 'รสชาติ']

In [78]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b:latest",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [79]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 156 ms, sys: 14.3 ms, total: 171 ms
Wall time: 15.4 s


In [80]:
results[0]['text']['keywords'], results[1]['text']['keywords']
(['ราคา', 'รสชาติ', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'บริการ'])

(['ราคา', 'รสชาติ', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'บริการ'])

In [81]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [82]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [83]:
inputs = [{"review": r} for r in sample_reviews]

In [84]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 17.2 ms, sys: 3.45 ms, total: 20.7 ms
Wall time: 2.42 s


In [85]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [86]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [87]:
from tqdm import tqdm

In [88]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [20:11<00:00,  9.70s/it]


In [89]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, บรรยากาศคลาสสิก
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [90]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [91]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [92]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [93]:
context_extract = context_prompt | llm3 | context_parser

In [94]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [95]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [96]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords']
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 93204.68it/s]


In [97]:
len(context_inputs)

24816

In [98]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:28:39<00:00,  5.32s/it]


In [99]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสว

In [100]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 434192.96it/s]


In [101]:
len(sentiment_inputs)

3000

In [102]:
sentiment_inputs[:6]

[{'message': 'ร้านอาหารใหญ่มากกกกกกก'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล'}]

In [103]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [43:06<00:00,  2.59s/it]


In [104]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'],
  'sentiment': ['neutral', 'neutral', 'negative']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยา

In [107]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

KeyError: 'keywords'

In [108]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')